In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.13.0'

In [ ]:
data = pd.read_csv("data.csv")
X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


**Data Preprocessing**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
tr = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])], remainder='passthrough')
X = np.array(tr.fit_transform(X))


In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Building ANN**

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
#Input layer and 1st hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation = "relu")) #fully connected hidden layer and input layer

In [ ]:
#adding 2nd hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation = "relu"))

In [ ]:
#output layer
ann.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid")) #binary dependent so units = 1

**Training**

In [ ]:
ann.compile(optimizer = 'adam' ,loss = 'binary_crossentropy' ,metrics =['accuracy'])

In [ ]:
#training the model
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5823 - accuracy: 0.7639
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4591 - accuracy: 0.7964
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.8096
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4026 - accuracy: 0.8328
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3858 - accuracy: 0.8434
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3746 - accuracy: 0.8453
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3673 - accuracy: 0.8469
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3636 - accuracy: 0.8500
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3603 - accuracy: 0.8528
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3584 - accura

**Evaluating the model and predictions**

In [ ]:
test_sample_1 = [[1, 0, 0, 600, 1, 40, 3, 55000, 2, 1, 1, 50000]]
print(ann.predict(sc.transform(test_sample_1)) > 0.50)
#False - customer does not leave
#True - customer leaves

1/1 [==============================] - 0s 44ms/step
[[False]]


In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
conf_matrix = confusion_matrix(y_test,y_pred)
print("confusion matrix : ", conf_matrix)
print("Accuracy of the model : ",accuracy_score(y_test, y_pred))

confusion matrix :  [[1542   53]
 [ 216  189]]
Accuracy of the model :  0.8655


Based on the accuracy score, we find that 86 % correct predictions are achieved by this model